In [8]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, LSTM, Dropout, Flatten, Bidirectional
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
!pip install -U -q PyDrive
from pydrive.drive import GoogleDrive
from google.colab import drive
# data_path = '/home/arsh/Jasleen/Spring 2023/NLP/Group Project/Authorship-Attribution-for-Neural-Text-Generation-master/data/'
from google.colab import drive
drive.mount('/content/drive')
data_path = 'drive/MyDrive/NLP/'
data = pd.read_csv(data_path+'input.csv')





Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
texts = data['text'].tolist()
classes = data['class'].tolist()

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

tokenized_texts = [word_tokenize(text.lower()) for text in texts]
stop_words = set(stopwords.words('english'))
punctuation = set(string.punctuation)
lemmatizer = WordNetLemmatizer()

preprocessed_texts = []
for tokens in tokenized_texts:
    filtered_tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words and token not in punctuation]
    preprocessed_texts.append(filtered_tokens)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_texts)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(preprocessed_texts)
max_sequence_length = 500
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

label_encoder = LabelEncoder()
encoded_classes = label_encoder.fit_transform(classes)
num_classes = len(set(classes))
one_hot_classes = to_categorical(encoded_classes, num_classes=num_classes)

X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_classes, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(len(word_index) + 1, 100, input_length=max_sequence_length))
model.add(Conv1D(filters=128, kernel_size=5, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(Bidirectional(LSTM(128)))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=128)

model2 = Sequential()
model2.add(Embedding(len(word_index) + 1, 100, input_length=max_sequence_length))
model2.add(LSTM(128))
model2.add(Dropout(0.2))
model2.add(Dense(num_classes, activation='softmax'))
model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model2.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=128)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Epoch 1/5
74/74 [==============================] - 27s 301ms/step - loss: 1.7215 - accuracy: 0.3769 - val_loss: 1.1390 - val_accuracy: 0.5708
Epoch 2/5
74/74 [==============================] - 17s 222ms/step - loss: 0.8077 - accuracy: 0.7023 - val_loss: 0.7997 - val_accuracy: 0.6931
Epoch 3/5
74/74 [==============================] - 18s 244ms/step - loss: 0.4507 - accuracy: 0.8350 - val_loss: 0.7356 - val_accuracy: 0.7323
Epoch 4/5
74/74 [==============================] - 22s 299ms/step - loss: 0.2355 - accuracy: 0.9203 - val_loss: 0.8772 - val_accuracy: 0.7310
Epoch 5/5
74/74 [==============================] - 13s 176ms/step - loss: 0.1032 - accuracy: 0.9706 - val_loss: 0.9572 - val_accuracy: 0.7374
Epoch 1/5
74/74 [==============================] - 19s 225ms/step - loss: 2.0323 - accuracy: 0.2243 - val_loss: 1.5750 - val_accuracy: 0.3815
Epoch 2/5
74/74 [==============================] - 17s 233ms/step - loss: 1.4413 - accuracy: 0.4275 - val_loss: 1.3832 - val_accuracy: 0.4774
Epoch 

In [11]:
cnn_lstm_predictions = model.predict(X_test)
lstm_rnn_predictions = model2.predict(X_test)

combined_predictions = []
for i in range(len(cnn_lstm_predictions)):
    combined_predictions.append(np.argmax(cnn_lstm_predictions[i] + lstm_rnn_predictions[i]))

combined_accuracy = accuracy_score(np.argmax(y_test, axis=1), combined_predictions)
print("Combined model accuracy:", combined_accuracy)

print("Classification Report:")
print(classification_report(np.argmax(y_test, axis=1), combined_predictions,target_names=label_encoder.classes_))

74/74 [==============================] - 1s 9ms/step
Combined model accuracy: 0.7502131287297528
Classification Report:
              precision    recall  f1-score   support

        ctrl       0.95      0.93      0.94       214
        fair       0.53      0.70      0.60       209
         gpt       1.00      1.00      1.00       217
        gpt2       0.64      0.60      0.62       213
        gpt3       0.48      0.55      0.51       217
      grover       0.56      0.56      0.56       222
       human       0.72      0.72      0.72       220
 instructgpt       0.82      0.51      0.63       202
        pplm       0.79      0.75      0.77       206
         xlm       0.98      0.97      0.97       215
       xlnet       0.92      0.97      0.94       211

    accuracy                           0.75      2346
   macro avg       0.76      0.75      0.75      2346
weighted avg       0.76      0.75      0.75      2346



In [ ]:
import pandas as pd
import numpy as np
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import string
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Embedding, Conv1D, MaxPooling1D, LSTM, Dropout, Flatten, Bidirectional
from keras.utils import pad_sequences
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Step 1: Load the data from the CSV file

data = pd.read_csv("/Users/charansaisadla/Downloads/input-2.csv")

texts = data['text'].tolist()
classes = data['class'].tolist()

# Step 2: Convert the text data into numerical representations
nltk.download('punkt')
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

tokenized_texts = []
for text in texts:
    tokens = [lemmatizer.lemmatize(token.lower()) for token in word_tokenize(text) if
              token.lower() not in stop_words and token.lower() not in string.punctuation]
    tokenized_texts.append(tokens)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_texts)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(tokenized_texts)
max_sequence_length = 500
padded_sequences = pad_sequences(sequences, maxlen=max_sequence_length)

# Step 3: Encode the class labels
label_encoder = LabelEncoder()
encoded_classes = label_encoder.fit_transform(classes)
num_classes = len(label_encoder.classes_)
one_hot_classes = to_categorical(encoded_classes, num_classes=num_classes)

# Step 4: Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(padded_sequences, one_hot_classes, test_size=0.2, random_state=42)

# Step 5: Define and train the CRNN model
model = Sequential()
model.add(Embedding(len(word_index) + 1, 100, input_length=max_sequence_length))
model.add(Conv1D(256, 8, activation='relu'))
model.add(MaxPooling1D(pool_size=5))
model.add(LSTM(128))
model.add(Dense(num_classes, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=5, batch_size=256)

# Step 6: Evaluate the model on the test set
y_pred_prob = model.predict(X_test)
y_pred = y_pred_prob.argmax(axis=1)
y_test_encoded = y_test.argmax(axis=1)
report = classification_report(y_test_encoded, y_pred)
print(report)


[nltk_data] Downloading package punkt to
[nltk_data]     /Users/charansaisadla/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/charansaisadla/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/5


2023-05-09 12:55:35.644279: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


37/37 [==============================] - 28s 745ms/step - loss: 1.8806 - accuracy: 0.3483 - val_loss: 1.1191 - val_accuracy: 0.6053
Epoch 2/5
37/37 [==============================] - 29s 794ms/step - loss: 0.8174 - accuracy: 0.7130 - val_loss: 0.7992 - val_accuracy: 0.7221
Epoch 3/5
37/37 [==============================] - 29s 785ms/step - loss: 0.4261 - accuracy: 0.8592 - val_loss: 0.7758 - val_accuracy: 0.7361
Epoch 4/5
37/37 [==============================] - 29s 785ms/step - loss: 0.1786 - accuracy: 0.9478 - val_loss: 0.8523 - val_accuracy: 0.7327
Epoch 5/5
74/74 [==============================] - 3s 41ms/step
              precision    recall  f1-score   support

           0       0.93      0.94      0.93       214
           1       0.50      0.78      0.61       209
           2       1.00      1.00      1.00       217
           3       0.74      0.37      0.49       213
           4       0.55      0.46      0.50       217
           5       0.49      0.60      0.54       222